In [1]:
#The segmentation results are the in the segmentation_results2 folder.
#This can be found on Rebecca's Github but also within the "cropping_images" folder

In [2]:
##First step is to create the subfolders. This makes it much easier for the pipeline analysis.

In [ ]:
import os


master_folder = r"C:\Users\smithm27\Documents\cm4ai\segmentation_results2"

png_mask_folder_cell   = os.path.join(master_folder, "png_cell_mask")
png_mask_folder_nuclei = os.path.join(master_folder, "png_nuclei_mask")
npy_mask_folder_cell   = os.path.join(master_folder, "npy_cell_prediction")
npy_mask_folder_nuclei = os.path.join(master_folder, "npy_nuclei_prediction")
overlay_save_folder    = os.path.join(master_folder, "overlays")

folders = [
    png_mask_folder_cell,
    png_mask_folder_nuclei,
    npy_mask_folder_cell,
    npy_mask_folder_nuclei,
    overlay_save_folder
]

# Create folders if they don't exist
for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"✅ Created folder: {folder}")

# -----------------------------
# Move files into correct folders
# -----------------------------
for f in os.listdir(master_folder):
    f_path = os.path.join(master_folder, f)
    
    if os.path.isfile(f_path):
        # PNG masks
        if f.endswith("_cell_mask.png"):
            os.rename(f_path, os.path.join(png_mask_folder_cell, f))
        elif f.endswith("_nuclei_mask.png"):
            os.rename(f_path, os.path.join(png_mask_folder_nuclei, f))
        
        # NPY predictions
        elif f.endswith("_cell_prediction.npy"):
            os.rename(f_path, os.path.join(npy_mask_folder_cell, f))
        elif f.endswith("_nuclei_prediction.npy"):
            os.rename(f_path, os.path.join(npy_mask_folder_nuclei, f))

print("✅ Files organized into subfolders successfully.")


In [3]:
#Now that those are sorted, the next step will overlay the masks of all 4 channels and save the images as png files in one big folder.

In [ ]:
import os
import numpy as np
import imageio
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
image_folder = r"C:\Users\smithm27\Documents\cm4ai\data"

# Mask folders
png_mask_folder_cell   = r"C:\Users\smithm27\Documents\cm4ai\cropping_image\png_cell_mask"
png_mask_folder_nuclei = r"C:\Users\smithm27\Documents\cm4ai\cropping_image\png_nuclei_mask"
npy_mask_folder_cell   = r"C:\Users\smithm27\Documents\cm4ai\cropping_image\npy_cell_prediction"
npy_mask_folder_nuclei = r"C:\Users\smithm27\Documents\cm4ai\cropping_image\npy_nuclei_prediction"

overlay_save_folder = r"C:\Users\smithm27\Documents\cm4ai\cropping_image\overlays"
os.makedirs(overlay_save_folder, exist_ok=True)

# -----------------------------
# Parameters
# -----------------------------
CROP_SIZE = 1024  # bigger window to show more cells
OUTPUT_SIZE = 512  # final saved image size

# -----------------------------
# Helper functions
# -----------------------------
def load_gray(path):
    img = imageio.imread(path)
    if img.ndim == 3:
        img = np.mean(img, axis=2)
    return img.astype(float)

def get_bbox(mask):
    """Find bounding box of nonzero region."""
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    if not rows.any() or not cols.any():
        return None  # empty mask
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax+1, cmin, cmax+1

def center_crop(img, center_r, center_c, size):
    """Crop square window around center (padded if out of bounds)."""
    h, w = img.shape[:2]
    half = size // 2
    r1 = max(center_r - half, 0)
    r2 = min(center_r + half, h)
    c1 = max(center_c - half, 0)
    c2 = min(center_c + half, w)
    crop = np.zeros((size, size) + img.shape[2:], dtype=img.dtype)
    rr = r2 - r1
    cc = c2 - c1
    crop[:rr, :cc] = img[r1:r2, c1:c2]
    return crop

# -----------------------------
# Overlay generator
# -----------------------------
def generate_overlay(base_stem, mask_type="png"):
    # Load channels
    try:
        R = load_gray(os.path.join(image_folder, "red", f"{base_stem}_red.jpg"))
        B = load_gray(os.path.join(image_folder, "blue", f"{base_stem}_blue.jpg"))
        Y = load_gray(os.path.join(image_folder, "yellow", f"{base_stem}_yellow.jpg"))
        G = np.zeros_like(R)
    except FileNotFoundError as e:
        print(f"⚠️ Missing image for {base_stem}: {e}")
        return

    # Load masks
    if mask_type == "png":
        cell_path   = os.path.join(png_mask_folder_cell, f"{base_stem}_cell_mask.png")
        nuclei_path = os.path.join(png_mask_folder_nuclei, f"{base_stem}_nuclei_mask.png")
        cell_mask   = load_gray(cell_path) > 0 if os.path.exists(cell_path) else np.zeros_like(R, dtype=bool)
        nuclei_mask = load_gray(nuclei_path) > 0 if os.path.exists(nuclei_path) else np.zeros_like(R, dtype=bool)
    else:  # npy
        cell_path   = os.path.join(npy_mask_folder_cell, f"{base_stem}_cell_prediction.npy")
        nuclei_path = os.path.join(npy_mask_folder_nuclei, f"{base_stem}_nuclei_prediction.npy")
        cell_mask   = np.load(cell_path) if os.path.exists(cell_path) else np.zeros_like(R)
        nuclei_mask = np.load(nuclei_path) if os.path.exists(nuclei_path) else np.zeros_like(R)
        if cell_mask.ndim == 3: cell_mask = cell_mask[...,0]
        if nuclei_mask.ndim == 3: nuclei_mask = nuclei_mask[...,0]
        cell_mask   = cell_mask > 0
        nuclei_mask = nuclei_mask > 0

    # Get union mask center
    union_mask = np.logical_or(cell_mask, nuclei_mask)
    bbox = get_bbox(union_mask)
    if bbox is None:
        print(f"⚠️ Empty mask for {base_stem}, using full image")
        center_r, center_c = R.shape[0]//2, R.shape[1]//2
    else:
        rmin, rmax, cmin, cmax = bbox
        center_r = (rmin + rmax)//2
        center_c = (cmin + cmax)//2

    # Crop fixed window
    Rc = center_crop(R, center_r, center_c, CROP_SIZE)
    Gc = center_crop(G, center_r, center_c, CROP_SIZE)
    Bc = center_crop(B, center_r, center_c, CROP_SIZE)
    Yc = center_crop(Y, center_r, center_c, CROP_SIZE)
    cell_crop   = center_crop(cell_mask.astype(float), center_r, center_c, CROP_SIZE)
    nuclei_crop = center_crop(nuclei_mask.astype(float), center_r, center_c, CROP_SIZE)

    # Normalize channels
    for arr in (Rc, Gc, Bc, Yc):
        arr -= arr.min()
        if arr.max() > 0:
            arr /= arr.max()

    # Compose RGB
    img_rgb = np.stack([Rc + Yc, Gc + Yc*0.5, Bc], axis=2)
    img_rgb = np.clip(img_rgb, 0, 1)

    # Overlay masks
    overlay = img_rgb.copy()
    overlay[...,0] = np.clip(overlay[...,0] + cell_crop*0.5, 0, 1)
    overlay[...,1] = np.clip(overlay[...,1] + nuclei_crop*0.5, 0, 1)
    overlay[...,2] = np.clip(overlay[...,2] + nuclei_crop*0.5, 0, 1)

    # Resize to output
    overlay_resized = np.array(
        plt.imshow(overlay)
    )

    # Show overlay
    plt.figure(figsize=(6,6))
    plt.imshow(overlay)
    plt.axis('off')
    plt.title(f"{base_stem} overlay ({mask_type})")
    plt.show()

    # Save overlay
    save_path = os.path.join(overlay_save_folder, f"{base_stem}_{mask_type}_overlay.png")
    imageio.imwrite(save_path, (overlay * 255).astype(np.uint8))
    print(f"✅ Saved overlay: {save_path}")

# -----------------------------
# Run for PNG masks
# -----------------------------
png_files = os.listdir(png_mask_folder_cell)
for f in png_files:
    if f.endswith("_cell_mask.png"):
        base_stem = f.replace("_cell_mask.png", "")
        generate_overlay(base_stem, mask_type="png")

# -----------------------------
# Run for NPY masks
# -----------------------------
npy_files = os.listdir(npy_mask_folder_cell)
for f in npy_files:
    if f.endswith("_cell_prediction.npy"):
        base_stem = f.replace("_cell_prediction.npy", "")
        generate_overlay(base_stem, mask_type="npy")

In [4]:
#Now to create the csv file for the next step of the pipeline

In [ ]:
import csv

csv_save_path = os.path.join(overlay_save_folder, "pred_cell.csv")

# Collect rows for CSV
csv_rows = []
csv_rows.append(["r_image","y_image","b_image","g_image","output_folder","output_prefix"])

# Re-scan PNG mask files (you could also reuse earlier lists)
for f in os.listdir(png_mask_folder_cell):
    if f.endswith("_cell_mask.png"):
        base_stem = f.replace("_cell_mask.png", "")
        r_image = os.path.join("images", f"{base_stem}_red.jpg")
        y_image = os.path.join("images", f"{base_stem}_yellow.jpg")
        b_image = os.path.join("images", f"{base_stem}_blue.jpg")
        g_image = os.path.join("images", f"{base_stem}_green.jpg")  # if missing, leave empty string ""
        output_folder = "output"
        output_prefix = base_stem + "_"
        csv_rows.append([r_image, y_image, b_image, g_image, output_folder, output_prefix])

# Write CSV
with open(csv_save_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(csv_rows)

print(f"✅ Saved CSV manifest: {csv_save_path}")

In [ ]:
#The results I generated are already saved in the folder labeled "cropping_results" along with the pred_cell.csv file